### Change Pickle data to JSON

It's probably a better idea to change your `pkl` format to JSON for mostly just two reasons -

* JSON files are human readable
* The natural structure of the data itself is much better suited to the JSON format

For that reason, I'll just convert your `data.pkl` to a JSON file and then save it first.

In [4]:
import pickle
import json

with open("data.pkl", "rb") as f:
    data = pickle.load(f)
    
home_team, away_team = data["home"]["name"], data["away"]["name"]
with open(f"{home_team}_{away_team}.json", "w") as f: ## save as Arsenal_Liverpool.json in present directory
    json.dump(data, f, indent=4)
    

### Passmap

The next few functions just plot the passmap. This is mostly my code for creating passmaps from the WhoScored data. There's a ton of room for improvements (varying thickness of the passes, better colors, better fonts, backgrounds, more network information to name a few) and frankly I haven't bothered to do any of that because I'm too lazy. 

But if you go through the code, I'm certain you'll figure out what does what. 

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pandas import json_normalize

from pitch import pitch
from adjustText import adjust_text
from matplotlib import cm

plt.style.use("custom_viz_dark")

class Player:

    def __init__(self, name, pid, x, y, total):
        self.name = name
        self.pid = pid
        self.x = x
        self.y = y
        self.total = total
        

def get_receiver(df):
    """ Get the receiver for successful open-play passes"""

    df[["receiver_id", "receiver_team_id"]] = df[["playerId", "teamId"]].shift(-1)
    df = df.query("(outcomeType_value == 1) & (type_displayName == 'Pass') & (receiver_team_id == teamId)").reset_index(drop=True)
    df.dropna(subset=["playerId", "receiver_id", "teamId", "receiver_team_id"], inplace=True)
    df[["playerId", "receiver_id", "teamId", "receiver_team_id"]] = df[["playerId", "receiver_id", "teamId", "receiver_team_id"]].astype(int)
    return df

def plot_passmap(file, ax, team_name, colormap):
    """ Our main driver function. Tons of room for improvement, feel free to play around
    """

    cmap = cm.get_cmap(colormap)
    offset = 2 ##change the x and y coordinates of passes between two same players so that there's no overlap
    ax = pitch(ax, height=100, width=100, lw=.6, line_color="white", circle_radius=3500)

    with open(file) as f:
        md = json.load(f)
    
    side = "home" if md["home"]["name"] == team_name else "away"  
    team_id = md[side]["teamId"]  
    st_players = {} ##get the starting 11 players
    
    for player in md[side]["players"]:
        if "isFirstEleven" in player:
            st_players.update({player["playerId"]: player["name"]})        

    df = json_normalize(md["events"], sep="_")
    pdf = get_receiver(df)

    pdf = pdf[pdf["qualifiers"].apply(lambda x:x not in ["Freekick", "GoalKick", "Corner"])] ##remove set-pieces
    pdf = pdf.query("teamId == @team_id").reset_index(drop=True) 
    pdf = pdf.dropna(axis=1, how="all")
    group1 = pdf.groupby(["playerId", "receiver_id"]).agg(count=("teamId", "size")).reset_index()
    group1 = group1.loc[group1["count"]>1]
    group1["count"] = group1["count"]/group1["count"].max()
    group2 = pdf.groupby(["playerId"]).agg(x=("x", "median"), y=("y", "median"), total=("teamId", "size")).reset_index()
    ##change median to mean to get the mean positions of the players
    
    
    stplist = list(st_players.keys()) ##list of starting players ids
    group1 = group1.query("(playerId in @stplist) & (receiver_id in @stplist)").reset_index(drop=True)
    group2 = group2.query("playerId in @stplist").reset_index(drop=True)
    group2["player_name"] = group2["playerId"].map(st_players)
    

    plr_objs = {}
    for i in group2.index:
        plr_objs[group2.loc[i, "playerId"]] = Player(group2.loc[i, "player_name"], group2.loc[i, "playerId"], group2.loc[i, "x"], group2.loc[i, "y"], group2.loc[i, "total"])

    for (_, (pid, rid, count)) in group1.iterrows():
        if abs(plr_objs[pid].x - plr_objs[rid].x) >= abs(plr_objs[pid].y - plr_objs[rid].y):
            ax.annotate("", xy=(plr_objs[rid].x, plr_objs[rid].y + offset if rid>pid else -1*offset), xytext=(plr_objs[pid].x, plr_objs[pid].y + offset if rid>pid else -1*offset),
                    arrowprops={"arrowstyle":"->", "connectionstyle":"arc3", "lw":count*5, "color":cmap(count), "alpha":count, "linestyle":"dotted" if count<.5 else "solid"})
        else: 
            ax.annotate("", xy=(plr_objs[rid].x + offset if rid>pid else -1*offset, plr_objs[rid].y), xytext=(plr_objs[pid].x + offset if rid>pid else -1*offset, plr_objs[pid].y),
                    arrowprops={"arrowstyle":"->", "connectionstyle":"arc3", "lw":count*5, "color":cmap(count), "alpha":count, "linestyle":"dotted" if count<.5 else "solid"})            

    texts = []
    for player_id, player_obj in plr_objs.items():
        ax.scatter(player_obj.x, player_obj.y, s=player_obj.total*3, color="red", ec="k", alpha=.8, zorder=10, marker="h")
        texts.append(ax.text(player_obj.x, player_obj.y, s=player_obj.name.split(" ")[-1], fontsize=10, rotation=270, zorder=12))
    adjust_text(texts)

    ax.set(xlim=(-0.2,100.2), ylim=(-0.2,100))
    ax.set_aspect(.8)
    
    ax.text(105, 100, s=team_name, rotation=270, fontsize=20, va="top")
    ax.text(101, 100, s=f"{md['home']['name']} vs {md['away']['name']}", rotation=270, 
            fontsize=15, va="top")
    
    
### Calling all our functions
if __name__ == "__main__":
    fig, ax = plt.subplots(figsize=(12, 8)) 
    team_name = "Liverpool"
    file = r"Arsenal_Liverpool.json"
    colormap = "Blues" ## probably a better idea to change all these to a params dict and pass it to plot_passmap
    plot_passmap(file, ax, team_name, colormap)
    #plt.show() ##if you're on the shell or something


OSError: 'custom_viz_dark' not found in the style library and input is not a valid URL or path; see `style.available` for list of available styles